<a href="https://colab.research.google.com/github/eespejoruiz/IndexNow/blob/main/Index_Now_%7C_Amazon_Andes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1) API Search Console Index Now


In [ ]:
!pip3 install git+https://github.com/antoineeripret/gsc_wrapper

  Cloning https://github.com/antoineeripret/gsc_wrapper to /tmp/pip-req-build-i1llh6la
  Running command git clone --filter=blob:none --quiet https://github.com/antoineeripret/gsc_wrapper /tmp/pip-req-build-i1llh6la
  Resolved https://github.com/antoineeripret/gsc_wrapper to commit 17186d74169c14f98e049df9e9746ca0b4d6d029
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 3.6 MB/s eta 0:00:00
  Created wheel for gscwrapper: filename=gscwrapper-0.0.7-py3-none-any.whl size=99

# 2) Instalo Librerías

In [ ]:
import gscwrapper
import pandas as pd
import numpy as np
from gscwrapper.account import Account

In [ ]:
!pip show gscwrapper

Name: gscwrapper
Version: 0.0.7
Summary: A simple wrapper for Google Search Console API for SEO data analysis.
Home-page: https://github.com/antoineeripret/gsc_wrapper
Author: Antoine Eripret
Author-email: antoine.eripret.dev@gmail.com
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: google-api-python-client, google-auth, google-auth-oauthlib, google.cloud, numpy, pandas, pandas-gbq, prophet, pycausalimpact, python-dateutil, requests, tqdm, validators
Required-by: 


# 3) Me conecto a google drive


In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 4) Me Conecto al API y Selecciono de Propiedad en Google Search Console


In [ ]:
account_service = (
    gscwrapper
    .generate_auth(
        client_config="/content/drive/MyDrive/000 | Amazon Andes/indexing-cs-amazon-andes.json",
        service_account_auth=True
    )
)

In [ ]:
import os
from google.oauth2 import service_account
from googleapiclient.discovery import build

# 📌 Ruta del archivo JSON de la cuenta de servicio en Google Drive
credenciales_json = "/content/drive/MyDrive/000 | Amazon Andes/indexing-cs-amazon-andes.json"

# 📌 Definir el `scope` correcto para la API de Indexing
SCOPES = ["https://www.googleapis.com/auth/indexing"]

def autenticar_con_cuenta_de_servicio():
    """Autentica con una cuenta de servicio sin intervención manual."""

    creds = service_account.Credentials.from_service_account_file(
        credenciales_json,
        scopes=SCOPES
    )

    return creds

# 🔹 Ejecutar la autenticación con la cuenta de servicio
credentials = autenticar_con_cuenta_de_servicio()

# 🔹 Crear servicio para la API de Indexing con los permisos correctos
service = build("indexing", "v3", credentials=credentials)

print("✅ Autenticación exitosa con cuenta de servicio. 'service' está listo para Indexing API.")

✅ Autenticación exitosa con cuenta de servicio. 'service' está listo para Indexing API.


In [ ]:
account_service

<searchconsole.account.Account>

# 5) Extraer URLs del Sitemap

In [ ]:
import requests
import xml.etree.ElementTree as ET
import os
import gzip
import io
import time

# 📌 URLs base
sitemap_index_url = "https://amazon-andes.com/sitemap_index.xml"
ruta_indexadas = "/content/drive/MyDrive/000 | Amazon Andes/urls_indexadas.txt"
ruta_por_indexar = "/content/drive/MyDrive/000 | Amazon Andes/urls_por_indexar.txt"

def obtener_sitemaps(url):
    """Obtiene la lista de archivos sitemap desde el sitemap index con reintentos en caso de fallo."""
    for intento in range(3):  # Reintentar hasta 3 veces
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()

            # Verificar si el contenido es XML válido
            if "xml" in response.headers.get("Content-Type", ""):
                root = ET.fromstring(response.content)
                sitemaps = [elem.text for elem in root.findall(".//{http://www.sitemaps.org/schemas/sitemap/0.9}loc")]
                return sitemaps
            else:
                print("❌ El contenido recibido no es XML válido.")
                return []

        except requests.exceptions.RequestException as e:
            print(f"⚠️ Error al obtener el sitemap index. Intento {intento + 1}/3: {e}")
            time.sleep(2)

    print("❌ No se pudo obtener el sitemap index después de 3 intentos.")
    return []

def obtener_urls_de_sitemaps(lista_sitemaps):
    """Escanea todos los sitemaps y extrae las URLs, descomprimiendo archivos .gz si es necesario."""
    urls_totales = set()

    for sitemap in lista_sitemaps:
        for intento in range(3):  # Reintentar hasta 3 veces en caso de error
            try:
                response = requests.get(sitemap, timeout=10)
                response.raise_for_status()

                # Si el archivo es .xml.gz, descomprimirlo antes de procesarlo
                if sitemap.endswith(".gz"):
                    with gzip.GzipFile(fileobj=io.BytesIO(response.content)) as f:
                        xml_content = f.read().decode("utf-8")
                else:
                    xml_content = response.content.decode("utf-8")

                # Validar si el XML es válido
                if not xml_content.strip().startswith("<"):
                    print(f"⚠️ {sitemap} no contiene XML válido.")
                    continue

                # Parsear el XML
                root = ET.fromstring(xml_content)
                urls = [elem.text for elem in root.findall(".//{http://www.sitemaps.org/schemas/sitemap/0.9}loc")]
                urls_totales.update(urls)
                break  # Salir del bucle si el intento fue exitoso

            except (requests.exceptions.RequestException, ET.ParseError) as e:
                print(f"⚠️ Error en intento {intento + 1}/3 con {sitemap}: {e}")
                time.sleep(2)  # Esperar antes de reintentar

    return urls_totales

def leer_urls_indexadas(ruta):
    """Lee las URLs ya indexadas desde el archivo."""
    if os.path.exists(ruta):
        with open(ruta, "r") as f:
            return set(f.read().splitlines())
    return set()

def guardar_urls_nuevas(ruta, urls_nuevas):
    """Guarda solo las nuevas URLs en el archivo (sobrescribe el archivo anterior)."""
    if urls_nuevas:
        with open(ruta, "w") as f:  # ⚠️ 'w' para sobrescribir el archivo anterior
            for url in urls_nuevas:
                f.write(url + "\n")
        print(f"✅ {len(urls_nuevas)} nuevas URLs guardadas en '{ruta}'.")
    else:
        print("⚠️ No hay nuevas URLs para agregar.")

def actualizar_urls_indexadas(ruta, urls_nuevas):
    """Agrega las nuevas URLs indexadas al archivo `urls_indexadas.txt`."""
    if urls_nuevas:
        with open(ruta, "a") as f:  # ⚠️ 'a' para agregar nuevas URLs sin borrar las anteriores
            for url in urls_nuevas:
                f.write(url + "\n")
        print(f"✅ {len(urls_nuevas)} URLs añadidas a '{ruta}' para futuras referencias.")

# 🔹 1️⃣ Obtener todos los archivos de sitemap desde el sitemap index
sitemaps_encontrados = obtener_sitemaps(sitemap_index_url)
print(f"🔍 Se encontraron {len(sitemaps_encontrados)} sitemaps.")

# 🔹 2️⃣ Extraer todas las URLs de los sitemaps
urls_extraidas = obtener_urls_de_sitemaps(sitemaps_encontrados)
print(f"✅ Total de URLs extraídas: {len(urls_extraidas)}")

# 🔹 3️⃣ Leer el archivo de URLs indexadas y filtrar las nuevas
urls_indexadas = leer_urls_indexadas(ruta_indexadas)
urls_nuevas = urls_extraidas - urls_indexadas

# 🔹 4️⃣ Guardar solo las nuevas URLs en `urls_por_indexar.txt` para ser enviadas
guardar_urls_nuevas(ruta_por_indexar, urls_nuevas)

# 🔹 5️⃣ Actualizar `urls_indexadas.txt` con las nuevas URLs enviadas
actualizar_urls_indexadas(ruta_indexadas, urls_nuevas)

🔍 Se encontraron 12 sitemaps.
✅ Total de URLs extraídas: 415
✅ 1 nuevas URLs guardadas en '/content/drive/MyDrive/000 | Amazon Andes/urls_por_indexar.txt'.
✅ 1 URLs añadidas a '/content/drive/MyDrive/000 | Amazon Andes/urls_indexadas.txt' para futuras referencias.


# 6) Enviar las URLs Nuevas a Google Search Console

In [ ]:
import time
import os

# 📌 Ruta del archivo donde están las URLs a indexar
ruta_urls = "/content/drive/MyDrive/000 | Amazon Andes/urls_por_indexar.txt"
ruta_errores = "/content/drive/MyDrive/000 | Amazon Andes/urls_fallidas.txt"

def leer_urls(ruta):
    """Lee las URLs desde un archivo y las devuelve como una lista."""
    if os.path.exists(ruta):
        with open(ruta, "r") as f:
            return [line.strip() for line in f.readlines() if line.strip()]
    return []

def enviar_url_para_indexacion(url):
    """Envía una URL a la API de Indexing de Google y muestra el estado del envío."""
    request_body = {
        "url": url,
        "type": "URL_UPDATED"
    }

    try:
        response = service.urlNotifications().publish(body=request_body).execute()

        # 📌 Verificar si la API respondió correctamente
        if "urlNotificationMetadata" in response:
            print(f"✅ URL enviada con éxito: {url}")
            print(f"🔹 Última actualización: {response['urlNotificationMetadata'].get('latestUpdate', 'No disponible')}\n")
        else:
            print(f"⚠️ URL enviada pero sin confirmación clara: {url}")
            print(f"🔹 Respuesta: {response}\n")

        return True  # Indica que la URL se envió correctamente

    except Exception as e:
        print(f"❌ Error al enviar {url}: {e}\n")
        return False  # Indica que hubo un error

def guardar_urls_fallidas(urls):
    """Guarda las URLs que no pudieron enviarse para reintentos futuros."""
    if urls:
        with open(ruta_errores, "w") as f:
            for url in urls:
                f.write(url + "\n")
        print(f"⚠️ {len(urls)} URLs con error guardadas en '{ruta_errores}'.")

# 🔹 Leer las URLs desde el archivo
urls_a_indexar = leer_urls(ruta_urls)

if not urls_a_indexar:
    print("⚠️ No hay URLs en el archivo para indexar.")
else:
    print(f"🚀 Enviando {len(urls_a_indexar)} URLs a Google Indexing API...")

    # 🔹 Lista de URLs que fallaron al enviarse
    urls_fallidas = []

    # 🔹 Enviar todas las URLs a Google Indexing API
    for url in urls_a_indexar:
        if not enviar_url_para_indexacion(url):
            urls_fallidas.append(url)

        time.sleep(2)  # 🔹 Pausa para evitar bloqueos

    # 🔹 Guardar las URLs con errores para reintento
    guardar_urls_fallidas(urls_fallidas)

    print("✅ Proceso de envío completado.")

🚀 Enviando 1 URLs a Google Indexing API...
✅ URL enviada con éxito: https://amazon-andes.com/product/muna-essential-oil-peru/
🔹 Última actualización: No disponible

✅ Proceso de envío completado.


# 🔺 Enviar las TODAS las URLs a Google Search Console | Usar solo en casos de cambios generales en la web.

In [ ]:
import os
import time

# Archivos de URLs
ruta_urls = "/content/drive/MyDrive/000 | Amazon Andes/urls_indexadas.txt"
ruta_errores = "/content/drive/MyDrive/000 | Amazon Andes/urls_fallidas.txt"

def leer_urls(ruta):
    """Lee las URLs desde un archivo."""
    if os.path.exists(ruta):
        with open(ruta, "r") as f:
            return [line.strip() for line in f.readlines() if line.strip()]
    return []

def guardar_urls_fallidas(urls):
    """Guarda las URLs que fallaron en un archivo para reintentos futuros."""
    if urls:
        with open(ruta_errores, "a") as f:
            for url in urls:
                f.write(url + "\n")
        print(f"⚠️ {len(urls)} URLs con error guardadas en '{ruta_errores}'.")

def enviar_url_para_indexacion(url):
    """Envía una URL a la API de Indexing de Google."""
    request_body = {
        "url": url,
        "type": "URL_UPDATED"
    }
    try:
        response = service.urlNotifications().publish(body=request_body).execute()
        print(f"✅ URL enviada: {url} - Respuesta: {response}")
        return True
    except Exception as e:
        print(f"❌ Error al enviar {url}: {e}")
        return False

# Leer todas las URLs desde el archivo
urls_a_indexar = leer_urls(ruta_urls)

# Filtrar para enviar solo 2000 por día
urls_a_indexar = urls_a_indexar[:2000]

# Lista de URLs que no pudieron enviarse
urls_fallidas = []

if urls_a_indexar:
    print(f"🔹 Enviando {len(urls_a_indexar)} URLs a Google Indexing API...")
    for url in urls_a_indexar:
        if not enviar_url_para_indexacion(url):
            urls_fallidas.append(url)
        time.sleep(2)  # Pausa para evitar bloqueos
else:
    print("⚠️ No hay URLs en el archivo para indexar.")

# Guardar las URLs con errores
guardar_urls_fallidas(urls_fallidas)

# 📌 Código: Reintentar el envío de URLs fallidas

In [ ]:
import os
import time

# Archivo con las URLs que fallaron en intentos anteriores
ruta_errores = "/content/drive/MyDrive/000 | Expreso/urls_fallidas.txt"

def leer_urls(ruta):
    """Lee las URLs desde un archivo."""
    if os.path.exists(ruta):
        with open(ruta, "r") as f:
            return [line.strip() for line in f.readlines() if line.strip()]
    return []

def guardar_urls_fallidas(urls):
    """Sobrescribe el archivo con las URLs que aún fallan."""
    with open(ruta_errores, "w") as f:
        for url in urls:
            f.write(url + "\n")
    print(f"⚠️ {len(urls)} URLs siguen fallando y se guardaron en '{ruta_errores}'.")

def enviar_url_para_indexacion(url):
    """Envía una URL a la API de Indexing de Google."""
    request_body = {
        "url": url,
        "type": "URL_UPDATED"
    }
    try:
        response = service.urlNotifications().publish(body=request_body).execute()
        print(f"✅ URL enviada: {url} - Respuesta: {response}")
        return True
    except Exception as e:
        print(f"❌ Error al enviar {url}: {e}")
        return False

# Leer las URLs fallidas
urls_fallidas = leer_urls(ruta_errores)

# Lista de URLs que sigan fallando después del reintento
urls_errores_nuevos = []

if urls_fallidas:
    print(f"🔹 Reintentando enviar {len(urls_fallidas)} URLs a Google Indexing API...")
    for url in urls_fallidas:
        if not enviar_url_para_indexacion(url):
            urls_errores_nuevos.append(url)  # Guardar solo las que siguen fallando
        time.sleep(2)  # Pausa para evitar bloqueos
else:
    print("⚠️ No hay URLs fallidas para reintentar.")

# Guardar las URLs que aún fallan
guardar_urls_fallidas(urls_errores_nuevos)

# 🟢 Verificar y Enviar a Indexar las URLs No Indexadas



In [ ]:
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client requests

In [ ]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

# 📌 Ruta del archivo JSON de la cuenta de servicio en Google Drive
credenciales_json = "/content/drive/MyDrive/000 | Amazon Andes/indexing-cs-amazon-andes.json"

# 📌 Definir los permisos (scopes) correctos
SCOPES = ["https://www.googleapis.com/auth/webmasters", "https://www.googleapis.com/auth/indexing"]

# 🔹 Autenticar con Google Search Console API
credentials = service_account.Credentials.from_service_account_file(credenciales_json, scopes=SCOPES)
service_gsc = build("searchconsole", "v1", credentials=credentials)
service_indexing = build("indexing", "v3", credentials=credentials)

# 🔹 Verificar autenticación exitosa
print("✅ Autenticación exitosa con Google Search Console API y Google Indexing API")

✅ Autenticación exitosa con Google Search Console API y Google Indexing API


In [ ]:
from googleapiclient.discovery import build
from google.oauth2 import service_account

# 📌 Ruta del archivo JSON de la cuenta de servicio en Google Drive
credenciales_json = "/content/drive/MyDrive/000 | Amazon Andes/indexing-cs-amazon-andes.json"

# 📌 Definir los permisos (scopes) correctos
SCOPES = ["https://www.googleapis.com/auth/webmasters.readonly"]

# 🔹 Autenticar con Google Search Console API
credentials = service_account.Credentials.from_service_account_file(credenciales_json, scopes=SCOPES)
service_gsc = build("searchconsole", "v1", credentials=credentials)

# 🔹 Obtener la lista de sitios que la cuenta de servicio puede acceder
sites_list = service_gsc.sites().list().execute()

# 🔹 Mostrar los sitios a los que tiene acceso
for site in sites_list.get("siteEntry", []):
    print(f"🔹 Acceso a: {site['siteUrl']} - Rol: {site['permissionLevel']}")

🔹 Acceso a: sc-domain:palosantopremium.com - Rol: siteOwner
🔹 Acceso a: sc-domain:amazon-andes.com - Rol: siteOwner


In [ ]:
import requests
import csv
import time
import xml.etree.ElementTree as ET

# 📌 URL del sitemap indexado
sitemap_index_url = "https://amazon-andes.com/sitemap_index.xml"

# 📌 Archivos de salida
output_csv = "/content/drive/MyDrive/000 | Amazon Andes/resultados_indexacion.csv"
urls_no_indexadas_txt = "/content/drive/MyDrive/000 | Amazon Andes/urls_no_indexadas.txt"

# 🔹 1️⃣ Obtener todos los sitemaps desde `sitemap-index.xml`
def obtener_sitemaps(url_sitemap_index):
    response = requests.get(url_sitemap_index)
    sitemaps = []

    if response.status_code == 200:
        root = ET.fromstring(response.content)
        for elem in root.findall(".//{http://www.sitemaps.org/schemas/sitemap/0.9}loc"):
            sitemaps.append(elem.text)
    else:
        print("❌ Error al obtener el sitemap indexado")

    return sitemaps

# 🔹 2️⃣ Obtener todas las URLs de cada sitemap
def obtener_urls_de_sitemaps(lista_sitemaps):
    urls_totales = []

    for sitemap in lista_sitemaps:
        response = requests.get(sitemap)
        if response.status_code == 200:
            root = ET.fromstring(response.content)
            for elem in root.findall(".//{http://www.sitemaps.org/schemas/sitemap/0.9}loc"):
                urls_totales.append(elem.text)
        else:
            print(f"❌ Error al obtener {sitemap}")

    return urls_totales

# 🔹 3️⃣ Consultar si una URL está indexada en Google Search Console
def verificar_indexacion(service_gsc, url, site_url):
    try:
        request = {
            "startDate": "2024-01-01",
            "endDate": "2025-02-16",
            "dimensions": ["page"],
            "dimensionFilterGroups": [{
                "filters": [{
                    "dimension": "page",
                    "operator": "equals",
                    "expression": url
                }]
            }],
            "rowLimit": 1
        }

        # 🔹 Consultar la API de GSC
        response = service_gsc.searchanalytics().query(siteUrl=site_url, body=request).execute()

        if "rows" in response and len(response["rows"]) > 0:
            return "✅ Indexada"
        else:
            return "❌ No indexada"

    except Exception as e:
        print(f"⚠️ Error con {url}: {e}")
        return "⚠️ Error"

# 🔹 4️⃣ Ejecutar la verificación de indexación
def analizar_indexacion():
    site_url = "sc-domain:amazon-andes.com"  # ⚠️ Usa la versión exacta registrada en GSC
    print(f"🔹 Obteniendo lista de sitemaps desde: {sitemap_index_url}")
    lista_sitemaps = obtener_sitemaps(sitemap_index_url)

    urls_sitemap = obtener_urls_de_sitemaps(lista_sitemaps)
    resultados = []
    urls_no_indexadas = []

    print(f"🔹 Verificando indexación de {len(urls_sitemap)} URLs en Google Search Console...")

    for url in urls_sitemap:
        estado = verificar_indexacion(service_gsc, url, site_url)
        resultados.append([url, estado])

        if estado == "❌ No indexada":
            urls_no_indexadas.append(url)

        time.sleep(2)  # 🔹 Pausa para evitar límites de la API

    # Guardar los resultados
    with open(output_csv, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["URL", "Estado de Indexación"])
        writer.writerows(resultados)

    with open(urls_no_indexadas_txt, "w") as f:
        for url in urls_no_indexadas:
            f.write(url + "\n")

    print("✅ Análisis de indexación completado.")

# 🔹 Ejecutar la verificación de indexación
analizar_indexacion()

🔹 Obteniendo lista de sitemaps desde: https://amazon-andes.com/sitemap_index.xml
🔹 Verificando indexación de 415 URLs en Google Search Console...
✅ Análisis de indexación completado.


In [ ]:
from googleapiclient.discovery import build
from google.oauth2 import service_account

# 📌 Ruta del archivo JSON de la cuenta de servicio en Google Drive
credenciales_json = "/content/drive/MyDrive/000 | Amazon Andes/indexing-cs-amazon-andes.json"

# 📌 Definir el permiso (scope) correcto
SCOPES = ["https://www.googleapis.com/auth/webmasters.readonly"]

# 🔹 Autenticar con Google Search Console API
credentials = service_account.Credentials.from_service_account_file(credenciales_json, scopes=SCOPES)
service_gsc = build("searchconsole", "v1", credentials=credentials)

# 🔹 Obtener la lista de sitios que la cuenta de servicio puede acceder
sites_list = service_gsc.sites().list().execute()

# 🔹 Mostrar los sitios a los que tiene acceso
print("🔹 La cuenta de servicio tiene acceso a:")
for site in sites_list.get("siteEntry", []):
    print(f" - {site['siteUrl']} | Permiso: {site['permissionLevel']}")

🔹 La cuenta de servicio tiene acceso a:
 - sc-domain:palosantopremium.com | Permiso: siteOwner
 - sc-domain:amazon-andes.com | Permiso: siteOwner


In [ ]:
import os
import time

# 📌 Archivos de entrada/salida
ruta_urls_no_indexadas = "/content/drive/MyDrive/000 | Amazon Andes/urls_no_indexadas.txt"
ruta_errores = "/content/drive/MyDrive/000 | Amazon Andes/urls_fallidas.txt"

def leer_urls_no_indexadas(ruta):
    """Lee las URLs no indexadas desde un archivo."""
    if os.path.exists(ruta):
        with open(ruta, "r") as f:
            return [line.strip() for line in f.readlines() if line.strip()]
    return []

def guardar_urls_fallidas(urls):
    """Guarda las URLs que fallaron en un archivo para reintentos futuros."""
    if urls:
        with open(ruta_errores, "w") as f:
            for url in urls:
                f.write(url + "\n")
        print(f"⚠️ {len(urls)} URLs con error guardadas en '{ruta_errores}'.")

def actualizar_archivo_urls(urls_exitosas):
    """Elimina del archivo las URLs que ya fueron indexadas correctamente."""
    if not os.path.exists(ruta_urls_no_indexadas):
        return

    urls_actuales = leer_urls_no_indexadas(ruta_urls_no_indexadas)
    nuevas_urls = list(set(urls_actuales) - set(urls_exitosas))  # 🔹 Filtrar URLs exitosas

    with open(ruta_urls_no_indexadas, "w") as f:
        for url in nuevas_urls:
            f.write(url + "\n")

    print(f"📄 Archivo actualizado: {len(nuevas_urls)} URLs restantes en '{ruta_urls_no_indexadas}'.")

def enviar_url_para_indexacion(url):
    """Envía una URL a la API de Indexing de Google."""
    request_body = {
        "url": url,
        "type": "URL_UPDATED"
    }
    try:
        response = service_indexing.urlNotifications().publish(body=request_body).execute()
        print(f"✅ URL enviada a indexar: {url}")
        return True
    except Exception as e:
        print(f"❌ Error al enviar {url}: {e}")
        return False

# 🔹 Leer URLs a indexar
urls_a_indexar = leer_urls_no_indexadas(ruta_urls_no_indexadas)
urls_fallidas = []
urls_indexadas = []

if urls_a_indexar:
    print(f"🔹 Enviando {len(urls_a_indexar)} URLs a Google Indexing API...")
    for url in urls_a_indexar:
        if enviar_url_para_indexacion(url):
            urls_indexadas.append(url)  # Guardamos las exitosas
        else:
            urls_fallidas.append(url)  # Guardamos las fallidas
        time.sleep(2)  # 🔹 Evitar bloqueos con la API

# 🔹 Actualizar archivos
guardar_urls_fallidas(urls_fallidas)
actualizar_archivo_urls(urls_indexadas)

# Corregir errores de Indexación

📌 PASO 1: Instalar Dependencias

In [ ]:
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client requests pandas

📌 PASO 2: Autenticación con Google Search Console API

In [ ]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

# 📌 Ruta del archivo JSON de la cuenta de servicio
credenciales_json = "/content/drive/MyDrive/000 | Amazon Andes/indexing-cs-amazon-andes.json"

# 📌 Definir el permiso para GSC
SCOPES_GSC = ["https://www.googleapis.com/auth/webmasters.readonly"]
credentials_gsc = service_account.Credentials.from_service_account_file(credenciales_json, scopes=SCOPES_GSC)

# 🔹 Autenticación con Google Search Console API
service_gsc = build("searchconsole", "v1", credentials=credentials_gsc)

print("✅ Autenticación exitosa con Google Search Console API")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/000 | Amazon Andes/indexing-cs-amazon-andes.json'

📌 PASO 3: Obtener y Analizar Errores de Indexación:
Este script extrae los errores de indexación de Google Search Console y los guarda en un CSV.

In [ ]:
import os
import datetime
import requests
import xml.etree.ElementTree as ET
from googleapiclient.errors import HttpError

# 📌 Definir la URL del Sitemap Index
sitemap_index_url = "https://amazon-andes.com/sitemap_index.xml"

# 📌 Directorio donde se guardarán los archivos de errores
output_dir = "/content/drive/MyDrive/000 | Amazon Andes/errores_indexacion/"
os.makedirs(output_dir, exist_ok=True)  # Crear la carpeta si no existe

def obtener_urls_sitemap(url_sitemap):
    """Obtiene todas las URLs de los sitemaps del sitio."""
    urls_totales = set()
    response = requests.get(url_sitemap)

    if response.status_code == 200:
        root = ET.fromstring(response.content)

        # 📌 Buscar archivos sitemap dentro del índice
        sitemaps = [elem.text for elem in root.findall(".//{http://www.sitemaps.org/schemas/sitemap/0.9}loc")]

        for sitemap in sitemaps:
            response_sitemap = requests.get(sitemap)
            if response_sitemap.status_code == 200:
                root_sitemap = ET.fromstring(response_sitemap.content)
                urls = [elem.text for elem in root_sitemap.findall(".//{http://www.sitemaps.org/schemas/sitemap/0.9}loc")]
                urls_totales.update(urls)
    else:
        print(f"❌ Error al obtener el sitemap indexado: {response.status_code}")

    return list(urls_totales)

def obtener_y_clasificar_errores():
    """Obtiene los errores de indexación usando Site Inspection API y los clasifica dinámicamente."""
    try:
        errores_clasificados = {}  # Diccionario para almacenar las URLs según su tipo de error

        # 🔹 Obtener las URLs desde el sitemap
        urls_a_inspeccionar = obtener_urls_sitemap(sitemap_index_url)
        print(f"🔍 Se encontraron {len(urls_a_inspeccionar)} URLs en el sitemap.")

        for url in urls_a_inspeccionar:
            try:
                request = {"inspectionUrl": url, "siteUrl": site_url}
                response = service_gsc.urlInspection().index().inspect(body=request).execute()

                # 📌 Extraer el estado de indexación
                index_status = response["inspectionResult"]["indexStatusResult"]
                motivo = index_status.get("indexingState", "Desconocido")

                # Si es un error nuevo, creamos una nueva lista para almacenarlo
                if motivo not in errores_clasificados:
                    errores_clasificados[motivo] = []

                # Agregar la URL a la categoría de error correspondiente
                errores_clasificados[motivo].append(url)

                print(f"🔍 {url} → {motivo}")

            except HttpError as e:
                print(f"⚠️ Error con {url}: {e}")

        # 🔹 Guardar los errores en archivos separados
        for motivo, urls in errores_clasificados.items():
            nombre_archivo = motivo.replace(" ", "_") + ".txt"  # Formato de nombre del archivo
            ruta_archivo = os.path.join(output_dir, nombre_archivo)

            with open(ruta_archivo, "w") as f:
                for url in urls:
                    f.write(url + "\n")

            print(f"✅ {len(urls)} URLs guardadas en '{ruta_archivo}'")

        return errores_clasificados

    except Exception as e:
        print(f"❌ Error al obtener los errores de indexación: {e}")
        return None

# 🔹 Ejecutar extracción y clasificación de errores en un solo paso
errores_clasificados = obtener_y_clasificar_errores()

🔍 Se encontraron 414 URLs en el sitemap.
❌ Error al obtener los errores de indexación: name 'site_url' is not defined


📌 PASO 4: Corregir Errores de Indexación

1️⃣ Corregir URLs 404

🔹 Si la página da 404, podemos:
	•	Revisar si la URL correcta está en el sitemap y redirigirla (301).
	•	Enviar un 410 si la URL ya no existe y no debe ser indexada.
	•	Reemplazar en el sitemap por una versión correcta.

In [ ]:
import requests
import xml.etree.ElementTree as ET
import time
from googleapiclient.discovery import build
from google.oauth2 import service_account

# 📌 Configurar credenciales de la API de Search Console
SCOPES = ["https://www.googleapis.com/auth/webmasters.readonly"]
CREDENTIALS_FILE = "/content/drive/MyDrive/000 | Amazon Andes/indexing-cs-amazon-andes.json"
SITE_URL = "sc-domain:amazon-andes.com"

# 📌 URLs base
SITEMAP_INDEX_URL = "https://amazon-andes.com/sitemap_index.xml"
WORDPRESS_API_URL = "https://amazon-andes.com/wp-json/redirection/v1/redirect"
WORDPRESS_USER = "admin"
WORDPRESS_PASSWORD = "wfi6 lkKP 3P86 eeoD yBaa 6xKx"

# 🔹 1️⃣ Obtener todas las URLs del sitemap
def obtener_urls_sitemap():
    """Obtiene todas las URLs de los sitemaps listados en el sitemap index."""
    urls_totales = set()

    response = requests.get(SITEMAP_INDEX_URL)
    if response.status_code == 200:
        root = ET.fromstring(response.content)
        sitemaps = [elem.text for elem in root.findall(".//{http://www.sitemaps.org/schemas/sitemap/0.9}loc")]

        for sitemap in sitemaps:
            response = requests.get(sitemap)
            if response.status_code == 200:
                root = ET.fromstring(response.content)
                urls = [elem.text for elem in root.findall(".//{http://www.sitemaps.org/schemas/sitemap/0.9}loc")]
                urls_totales.update(urls)

    return urls_totales

# 🔹 2️⃣ Obtener URLs con error 404 desde Google Search Console
def obtener_urls_404():
    """Obtiene las URLs reportadas como 'No se ha encontrado (404)' en Google Search Console."""
    credentials = service_account.Credentials.from_service_account_file(CREDENTIALS_FILE, scopes=SCOPES)
    service = build("webmasters", "v3", credentials=credentials)

    query = {
        "startDate": "2024-01-01",
        "endDate": "2025-02-19",
        "dimensions": ["page"],
        "dimensionFilterGroups": [{
            "filters": [{
                "dimension": "page",
                "operator": "contains",
                "expression": "404"
            }]
        }],
        "rowLimit": 1000
    }

    try:
        response = service.searchanalytics().query(siteUrl=SITE_URL, body=query).execute()
        if "rows" in response:
            return {row["keys"][0] for row in response["rows"]}
        else:
            return set()
    except Exception as e:
        print(f"⚠️ Error al obtener las URLs 404 desde GSC: {e}")
        return set()

# 🔹 3️⃣ Comparar ambas listas y generar redirecciones
def agregar_redirecciones_en_wordpress():
    """Compara URLs del sitemap y GSC y crea redirecciones 301 en WordPress."""
    print("🔍 Obteniendo URLs del sitemap...")
    urls_sitemap = obtener_urls_sitemap()
    print(f"✅ {len(urls_sitemap)} URLs extraídas del sitemap.")

    print("🔍 Obteniendo URLs con error 404 desde GSC...")
    urls_404 = obtener_urls_404()
    print(f"❌ {len(urls_404)} URLs con error 404 encontradas en GSC.")

    urls_para_redirigir = urls_404 - urls_sitemap
    print(f"🔁 {len(urls_para_redirigir)} URLs serán redirigidas a la página principal.")

    if urls_para_redirigir:
        for url in urls_para_redirigir:
            data = {
                "source": url.replace(SITE_URL, ""),  # WordPress guarda rutas relativas
                "target": SITE_URL,
                "type": "301",
                "match": "url",
                "status": "enabled"
            }

            response = requests.post(WORDPRESS_API_URL, json=data, auth=(WORDPRESS_USER, WORDPRESS_PASSWORD))

            if response.status_code == 201:
                print(f"✅ Redirección añadida: {url} ➝ {SITE_URL}")
            else:
                print(f"⚠️ Error al agregar {url}: {response.text}")

    else:
        print("⚠️ No hay URLs nuevas para redirigir.")

# 🔹 Ejecutar el script
agregar_redirecciones_en_wordpress()

🔍 Obteniendo URLs del sitemap...
✅ 415 URLs extraídas del sitemap.
🔍 Obteniendo URLs con error 404 desde GSC...
❌ 1 URLs con error 404 encontradas en GSC.
🔁 1 URLs serán redirigidas a la página principal.
⚠️ Error al agregar https://ru.amazon-andes.com/product/tocosh-capsules-100-x-400-mg/?add-to-cart=404: {"code":"invalid_username","message":"<strong>Error:<\/strong> Unknown username. Check again or try your email address.","data":{"status":401}}


In [ ]:
import csv
import time
from googleapiclient.discovery import build
from google.oauth2 import service_account

# 📌 Configuración de la API de GSC
SCOPES = ["https://www.googleapis.com/auth/webmasters.readonly"]
CREDENTIALS_FILE = "/content/drive/MyDrive/000 | Amazon Andes/indexing-cs-amazon-andes.json"
SITE_URL = "sc-domain:amazon-andes.com"  # ⚠️ Usa la URL exacta registrada en GSC

# 📌 Archivo CSV de salida
OUTPUT_CSV = "/content/drive/MyDrive/000 | Amazon Andes/errores_404.csv"

# 🔹 1️⃣ Conectar con Google Search Console API
def conectar_gsc():
    """Autentica con la API de Google Search Console y devuelve el servicio."""
    credentials = service_account.Credentials.from_service_account_file(CREDENTIALS_FILE, scopes=SCOPES)
    return build("webmasters", "v3", credentials=credentials)

# 🔹 2️⃣ Obtener las URLs con error 404 en "Cobertura"
def obtener_urls_404():
    """Consulta Google Search Console y obtiene las URLs con error 404."""
    service = conectar_gsc()

    try:
        query = {
            "startDate": "2024-01-01",  # Fecha de inicio del análisis
            "endDate": "2024-12-31",    # Fecha de fin del análisis
            "dimensions": ["page"],
            "dimensionFilterGroups": [{
                "filters": [{
                    "dimension": "page",
                    "operator": "contains",
                    "expression": "404"
                }]
            }],
            "rowLimit": 1000  # Máximo de URLs a obtener en una consulta
        }

        response = service.searchanalytics().query(siteUrl=SITE_URL, body=query).execute()

        urls_404 = [row["keys"][0] for row in response.get("rows", [])]
        return urls_404

    except Exception as e:
        print(f"⚠️ Error al obtener las URLs 404 desde GSC: {e}")
        return []

# 🔹 3️⃣ Guardar los errores en un archivo CSV
def guardar_csv(urls_404):
    """Guarda las URLs con error 404 en un archivo CSV."""
    if urls_404:
        with open(OUTPUT_CSV, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(["URL 404"])  # Encabezado
            for url in urls_404:
                writer.writerow([url])
        print(f"✅ {len(urls_404)} URLs 404 guardadas en '{OUTPUT_CSV}'.")
    else:
        print("✅ No se encontraron errores 404 en GSC.")

# 🔹 Ejecutar el script
urls_404 = obtener_urls_404()
guardar_csv(urls_404)

✅ 1 URLs 404 guardadas en '/content/drive/MyDrive/000 | Amazon Andes/errores_404.csv'.


In [ ]:
import requests

def corregir_errores_404(df_errores):
    """Revisa URLs con error 404 y aplica soluciones"""
    urls_404 = df_errores[df_errores["Motivo"] == "No se ha encontrado (404)"]["URL"]

    for url in urls_404:
        print(f"🔍 Revisando: {url}")

        # 🔹 Verificar si la URL correcta existe en el sitemap
        response = requests.get(url)
        if response.status_code == 200:
            print(f"✅ La URL {url} ahora es accesible. Enviar a indexar.")
            enviar_url_para_indexacion(url)  # Función definida en el Script 2
        else:
            print(f"⚠️ La URL {url} sigue sin existir. Considera un redireccionamiento 301 o 410.")

# Ejecutar corrección de 404
corregir_errores_404(df_errores)

KeyError: 'Motivo'

2️⃣ Corregir Errores de Canónicas Incorrectas

🔹 Si la página tiene un rel=canonical incorrecto:
	•	Actualizar el rel=canonical en el HTML de la página afectada.
	•	Asegurar que la URL canónica apunta a la página correcta.

In [ ]:
def corregir_errores_canonicas(df_errores):
    """Revisa URLs con problemas de etiqueta canónica"""
    urls_canonicas = df_errores[df_errores["Motivo"] == "Página alternativa con etiqueta canónica adecuada"]["URL"]

    for url in urls_canonicas:
        print(f"🔍 Revisando: {url}")
        # Aquí se podría hacer un request y validar el contenido de <link rel="canonical">
        # Pero lo ideal es revisar manualmente en los archivos fuente del sitio
        print(f"⚠️ Verificar manualmente la etiqueta canonical en: {url}")

# Ejecutar revisión de etiquetas canónicas
corregir_errores_canonicas(df_errores)

3️⃣ Corregir Bloqueos 4xx

🔹 Si la página está bloqueada por robots.txt, 403, 401 o 410:
	•	Revisar las reglas en robots.txt.
	•	Quitar restricciones innecesarias en el .htaccess o server.conf.
	•	Permitir el acceso a Googlebot en la configuración del servidor.

In [ ]:
def corregir_errores_4xx(df_errores):
    """Revisa URLs bloqueadas por errores 4xx"""
    urls_bloqueadas = df_errores[df_errores["Motivo"].str.contains("4xx")]["URL"]

    for url in urls_bloqueadas:
        print(f"🔍 Revisando acceso a: {url}")
        response = requests.get(url)

        if response.status_code == 200:
            print(f"✅ La URL {url} ahora es accesible. Enviar a indexar.")
            enviar_url_para_indexacion(url)  # Función definida en el Script 2
        else:
            print(f"⚠️ La URL {url} sigue bloqueada. Revisar robots.txt o restricciones del servidor.")

# Ejecutar corrección de bloqueos 4xx
corregir_errores_4xx(df_errores)

📌 PASO 5: Enviar URLs Corregidas a Google Indexing API

Después de solucionar los problemas, podemos reenviar las URLs corregidas a Google Indexing API para que sean reindexadas más rápido.

🔹 Usa el Script 2 que ya creamos para enviar URLs a Google Indexing API.

In [ ]:
for url in df_errores["URL"]:
    enviar_url_para_indexacion(url)